In [1]:
# pip install lxml
# pip install -U finance-datareader

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
# url 설정하면 날짜 수정해서 가져올수있음!!
url = 'https://consensus.hankyung.com/analysis/list?sdate=2018-06-22&edate=2023-06-22&now_page=1&search_value=REPORT_TITLE&report_type=CO&pagenum=20&search_text=&business_code='
html = requests.get(url, headers={'User-Agent':'Gils'}).content
soup = BeautifulSoup(html, 'lxml')

In [4]:
# soup을 이용하여 table찾기 + 찾은 요소 저장
table = soup.find('div', {'class':'table_style01'}).find('table')

In [5]:
# 각 셀들이 th에 있음
table.find("tr")

<tr>
<th class="first"><a href="javascript:_Order_Link(10000000,1000000,'/analysis/list?&amp;sdate=2018-06-22&amp;edate=2023-06-22&amp;report_type=CO&amp;order_type=&amp;now_page=1')">작성일 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th><a href="javascript:_Order_Link(10000000,10000,'/analysis/list?&amp;sdate=2018-06-22&amp;edate=2023-06-22&amp;report_type=CO&amp;order_type=&amp;now_page=1')">제목 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th><a href="javascript:_Order_Link(10000000,10,'/analysis/list?&amp;sdate=2018-06-22&amp;edate=2023-06-22&amp;report_type=CO&amp;order_type=&amp;now_page=1')">적정가격 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th><a href="javascript:_Order_Link(10000000,1,'/analysis/list?&amp;sdate=2018-06-22&amp;edate=2023-06-22&amp;report_type=CO&amp;order_type=&amp;now_page=1')">투자의견 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th><a href="javascript:_Order_Link(10000000,1000,'/analysis/list?&amp;sdate=2018-06-22&amp;edate=2023-06-22&

In [6]:
table.find_all("tr")[1]

<tr class="first">
<td class="first txt_number">2023-06-22</td>
<td class="text_l">
<a href="/analysis/downpdf?report_idx=622103" onmouseout="$('#content_622103').hide();" onmouseover="$('#content_622103').show();" target="_blank">CJ제일제당(097950) 하반기 저점 통과 예상</a>
<div class="layerPop">
<div class="pop01 disNone" id="content_622103">
<strong>CJ제일제당(097950) 하반기 저점 통과 예상</strong>
<ul>
<li>하반기 저점 통과 예상 </li>
</ul>
</div>
</div>
</td>
<td class="text_r txt_number">420,000</td>
<td>
Buy                                </td>
<td>한유정</td>
<td>한화투자증권</td>
<td>
<div class="dv_input">
<!-- <a href="javascript:_popup_open('http://media.kisline.com/highlight/mainHighlight.nice?paper_stock=097950&nav=1',1000,800,'yes')"><img src="/images/btn_info.gif" alt="기업정보" /></a> -->
<a href="javascript:_popup_open('https://comp.kisline.com/hi/HI0100M010GE.nice?stockcd=097950&amp;nav=1',1220,800,'yes')"><img alt="기업정보" src="/images/btn_info.gif"/></a>
</div>
</td>
<td>
<div class="dv_input">
<a class="chart_btn"

In [7]:
table.find_all('tr')[2].find_all('td')[1].text

'\n서진시스템(178320) 글로벌 ESS의 고성장을 따라\n\n\n서진시스템(178320) 글로벌 ESS의 고성장을 따라\n\n글로벌 ESS의 고성장을 따라 \n\n\n\n'

In [8]:
# 제목에서 종목코드와 종목명을 추출 -> 노이즈를 제거
import FinanceDataReader as fdr
stocks = fdr.StockListing('KRX')
stocks.head()

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,71300,1,800,1.13,70200,71500,70100,11408738,811276776480,425645495815000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,569000,1,11000,1.97,556000,575000,556000,319114,181095007400,133146000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,114000,2,-1100,-0.96,114200,115900,113400,2610558,298204228604,82992269610000,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,756000,2,-1000,-0.13,756000,762000,753000,49120,37188627000,53807544000000,71174000,STK
4,051910,KR7051910008,LG화학,KOSPI,,721000,2,-2000,-0.28,721000,728000,710000,145825,105020924000,50897079303000,70592343,STK


In [9]:
stocks['Code'] = stocks['Code'].astype(str)
import re

def remove_noise_and_split_title(title):
    in_code = ''
    in_name = ''
    
    for code, name in stocks[['Code','Name']].values:
        if code in title and name in title:
            in_code = code
            in_name = name
    clean_title = re.sub('[^A-Za-z0-9가-힣]', ' ', title)
    
    clean_title = clean_title.replace(in_code, ' ')
    clean_title = clean_title.replace(in_name, ' ')
    while ' ' * 2 in clean_title:
        clean_title = clean_title.replace(' ' * 2, ' ')
        
    if in_name == '':
        return [None]
    else:
        return [in_name, in_code, clean_title]

In [10]:
data = []
for tr in table.find_all('tr')[1:]:
    record = []
    try:
        for i, td in enumerate(tr.find_all('td')[:6]):
            if i == 1:
                record += remove_noise_and_split_title(td.text)
            elif i == 3:
                record.append(td.text.replace(' ','').replace('\r','').replace('\n',''))
            else:
                record.append(td.text)
                
        data.append(record)
    except:
        pass

In [11]:
report_dt = pd.DataFrame(data, columns=['작성일', '종목별', '종목코드', '제목', '적정가격', '평가의견', '작성자', '작성기관'])

In [12]:
#데이터 프레임 저장
import datetime
now = datetime.datetime.now() 
report_dt.to_csv('.\\DATA\\hankyung_report_{}.csv'.format(now.strftime('%Y%m%d_%H%M%S')),encoding='utf-8-sig',index=False)

In [13]:
report_dt

,작성일,종목별,종목코드,제목,적정가격,평가의견,작성자,작성기관
0,2023-06-22,CJ제일제당,097950,하반기 저점 통과 예상 하반기 저점 통과 예상 하반기 저점 통과 예상,"420,000",Buy,한유정,한화투자증권
1,2023-06-22,서진시스템,178320,글로벌 ESS의 고성장을 따라 글로벌 ESS의 고성장을 따라 글로벌 ESS의 고성...,0,NotRated,이수림,유안타증권
2,2023-06-22,LG전자,066570,여전히 유효한 방향성 여전히 유효한 방향성 2Q23 Preview 눈높이 충족 예...,"165,000",Buy,양승수,메리츠증권
3,2023-06-21,오리온,271560,5월 잠정 실적 Review 5월 잠정 실적 Review 3개월 연속 영업이익률 개선,"170,000",Buy,오지우,이베스트증권
4,2023-06-21,현대차,005380,CEO Investor Day CEO Investor Day 중장기 투자계획 15...,"290,000",Buy,이상현,IBK투자증권
5,2023-06-21,코오롱인더,120110,수소에 이어 배터리로 사업 다각화 진행 중 수소에 이어 배터리로 사업 다각화 진행...,"65,000",Buy,이동욱,IBK투자증권
6,2023-06-21,오리온,271560,위기는 곧 기회 위기는 곧 기회 기대치에 부합할 2분기 High Base는 맞지만...,"180,000",Buy,한유정,한화투자증권
7,2023-06-21,씨앤씨인터내셔널,352480,그칠줄 모르는 성장세 그칠줄 모르는 성장세 그칠줄 모르는 성장세,"66,000",Buy,한유정,한화투자증권
8,2023-06-21,SK하이닉스,000660,예상보다 빠른 수익성 개선 예상보다 빠른 수익성 개선 예상보다 빠른 수익성 개선,"150,000",Buy,김광진,한화투자증권
9,2023-06-21,롯데케미칼,011170,버티는 사업자가 위너 Winner 인가 버티는 사업자가 위너 Winner 인가 국...,"220,000",Buy,노우호,메리츠증권
